In [1]:
from src import utils
import numpy as np
import pandas as pd
import dill
from importlib import reload

2025-08-06 12:54:10.321784: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-06 12:54:10.578877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754477650.837795 4002610 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754477650.887772 4002610 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754477651.094504 4002610 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
reload(utils)
df = utils.Extract_and_Save_from_PDB(input_file='/scratch-scc/users/u14286/piplines/Bind/data/large/alphafold_db/test_data/AF-P11388-F1-model_v4.pdb',
                                 from_dill=False,
                                saving_dir='tmp/', k_nearest=5, inteacting_residues=False,
                              un_dn=False, outtype='dill', save_file=False)

print(df)

#### 1- Extract for id AF-P11388-F1-model_v4
PDB State AF-P11388-F1-model_v4
#### 3- Structure cleaned and renumbered AF-P11388-F1-model_v4


sphere_mange_arete: inconcistence


Append time: 4.76837158203125e-07 Iter time: 0.537541389465332 Before centroid calc: 0.5374197959899902 SASA time: 4.0531158447265625e-06


In [5]:
from biopandas.pdb import PandasPdb as ppdb
from Bio.PDB.ResidueDepth import ResidueDepth, get_surface, min_dist, residue_depth
import gc
import os
import time
reload(utils)

pdbfile = ppdb().read_pdb(path='/scratch-scc/users/u14286/piplines/Bind/tmp/AF-P41516-F1-model_v4_A_gZK.pdb')
print(pdbfile.df.keys)
dfpd = pd.DataFrame(pdbfile.df['ATOM'])
for col in dfpd.columns:
    print(col, dfpd[col].tolist()[0], type(dfpd[col].tolist()[0]))




<built-in method keys of dict object at 0x14ccc0d17a00>
record_name ATOM <class 'str'>
atom_number 1 <class 'int'>
blank_1  <class 'str'>
atom_name N <class 'str'>
alt_loc  <class 'str'>
residue_name MET <class 'str'>
blank_2  <class 'str'>
chain_id A <class 'str'>
residue_number 1 <class 'int'>
insertion  <class 'str'>
blank_3  <class 'str'>
x_coord 36.12 <class 'float'>
y_coord 27.654 <class 'float'>
z_coord -86.89 <class 'float'>
occupancy 1.0 <class 'float'>
b_factor 31.27 <class 'float'>
blank_4  <class 'str'>
segment_id  <class 'str'>
element_symbol N <class 'str'>
charge nan <class 'float'>
line_idx 0 <class 'int'>


In [2]:
from src import utils, constants
import numpy as np
import pandas as pd
import dill
with open('/scratch-scc/users/u14286/piplines/Bind/tmp/AF-Q9VT52-F1-model_v4.dill', 'rb') as f:
    data = dill.load(f)

In [3]:
df = data['df']
df.columns

Index(['model', 'chain', 'residue', 'resname', 'x', 'y', 'z', 'element',
       'atom_name', 'aid', 'res_id', 'sidechain_pseudopos_x',
       'sidechain_pseudopos_y', 'sidechain_pseudopos_z',
       'backbone_pseudopos_x', 'backbone_pseudopos_y', 'backbone_pseudopos_z',
       'bfactor', 'sasa', 'rd_value', 'cx', 'cx_avg', 'cx_max', 'cx_min',
       'cx_std', 'u12_dist_4', 'u13_dist_4', 'u14_dist_4', 'u15_dist_4',
       'u16_dist_4', 'u17_dist_4', 'u18_dist_4', 'u19_dist_4',
       'nearest_neighbours_cb_4', 'nearest_distances_cb_4', 'u20_dist_4',
       'u21_dist_4', 'u22_dist_4', 'u23_dist_4', 'u25_dist_4', 'u26_dist_4',
       'u27_dist_4', 'u28_dist_4', 'nearest_neighbours_ca_4',
       'nearest_distances_ca_4', 't12_cos_4', 't23_cos_4', 't45_cos_4',
       't56_cos_4', 't78_cos_4', 't89_cos_4', 't108_cos_4', 't811_cos_4',
       't110_cos_4', 't311_cos_4', 't28_cos_4', 't47_cos_4', 't69_cos_4',
       't75_cos_4', 't95_cos_4', 't42_cos_4', 't62_cos_4', 't25_cos_4',
       't85_co

In [59]:
from src import utils, constants
from src.constants import one_hot_encoding_scalar

def extract_data_for_tfrecord(df: pd.DataFrame, pdb_id: str):
    '''
    Extracts data from the DataFrame for TFRecord format.
    Args:
        df (pd.DataFrame): DataFrame containing the data.
        pdb_id (str): PDB ID to be used for generating IDs.
    Returns:
        geo_arr (np.ndarray): Array containing the geometric and physiochemical features.
        labels_arr (np.ndarray): Array containing the labels (resname one-hot encoded).
        IDs (np.ndarray): Array containing the IDs for each residue.
        columns (list): List of column names for the features.'''
    # physiochem
    physiochem_arr = []
    for atom in ['N', 'CA', 'C', 'O']:
        dfsub = df[df['atom_name']==atom]
        arrsubset = dfsub[['sasa', 'cx', 'rd_value']].to_numpy() #(N, 3)
        physiochem_arr.append(arrsubset)
    physiochem_arr = np.concatenate(physiochem_arr, axis=-1) #(N, 12) float
    # labels (resname ohe scalar)
    labels_arr = np.array([[one_hot_encoding_scalar[key]] for key in dfsub.resname.tolist()]) #(N,1) float
    # plddt
    plddt_arr = dfsub.bfactor.to_numpy()[:, np.newaxis] #(N,1)
    # geometric
    geo_arr = dfsub.iloc[:, 25:].to_numpy() #(N, d) float
    columns = ['sasa_N', 'cx_N', 'rd_value_N',
               'sasa_CA', 'cx_CA', 'rd_value_CA',
               'sasa_C', 'cx_C', 'rd_value_C',
               'sasa_O', 'cx_O', 'rd_value_O', 'bfactor'] + list(dfsub.columns[25:])
    # concatenate all
    all_arr = np.concatenate([physiochem_arr,  plddt_arr, geo_arr], axis=-1) #(N, 12 + 1  + d)
    # generate IDs
    IDs = np.array([f"{pdb_id}_{i}" for i in range(len(dfsub))])[:, np.newaxis]  #(N, 1)
    return all_arr, labels_arr, IDs, columns

arr, labels_arr, IDs, columns = extract_data_for_tfrecord(df, 'abs')
# check dimensions
print(arr.shape)  # Should be (N, 12 + 1 + d)
print(labels_arr.shape)  # Should be (N, 1)
print(IDs.shape)  # Should be (N, 1)


(120, 86)
(120, 1)
(120, 1)


In [100]:
from importlib import reload
reload(utils)
tfmanager = utils.TFRecordManager(tfrecord_path='tmp/test.tfrecord', feature_dim=86)
tfmanager.write_samples(features=arr, labels=labels_arr, ID_arrs=IDs)
train_ds = tfmanager.read_dataset()

(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1,) (1,)
(86,) (1

In [109]:
for i, (features, label, ID) in enumerate(train_ds.take(5)):  # Take first 5 samples
    print(f"Sample {i}:")
    print("Features:", features.numpy()[0])     # Shape: (feature_dim,)
    print("Label:", label.numpy()[0])           # Shape: (1,)
    print("ID:", ID.numpy()[0])        # It's a byte string


Sample 0:
Features: [ 2.5380212e-01  3.8405973e-03  2.9525682e-01  0.0000000e+00
  5.1270709e-03  2.6661521e-01  8.3750114e-02  4.0705688e-03
  2.4106996e-01  2.2956748e-01  3.8405973e-03  1.9905294e-01
  6.9500000e+01  9.8938570e+00  5.7832665e+00  1.0757808e+01
  1.3686677e+01  1.3592615e+01  1.5657337e+01  1.1993839e+01
  1.7055828e+01  4.5000000e+01  9.2038164e+00  9.3617983e+00
  8.9843407e+00  6.0637279e+00  9.9010267e+00  5.6090851e+00
  1.1690099e+01  1.0658095e+01  7.6967077e+00  1.1100000e+02
  8.6440926e+00 -2.7891454e-01 -6.4973813e-01 -7.7792984e-01
  1.7694494e-01 -5.3835064e-01 -7.2398186e-01 -4.6356475e-01
 -3.6036691e-01  2.0652121e-01 -6.6749585e-01  2.5284791e-01
  4.4891387e-01 -6.2907344e-01 -6.1247790e-01 -5.8114994e-01
 -9.1928619e-01 -6.9550771e-01  5.6932461e-01  9.3704754e-01
 -9.7754437e-01  2.1072967e-01  1.8352903e-01  9.8301429e-01
  6.8317211e-01 -7.3025745e-01  7.6354522e-01 -6.4575434e-01
 -9.7259295e-01 -2.3251437e-01  4.0000000e+00  1.7917595e+00
 -4.

2025-08-05 18:18:09.787838: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [1]:
df

NameError: name 'df' is not defined